# This is the notebook used to generate `census_distribution.txt`

In [ ]:
import nationstates as ns
import trotterdam as t

from IPython.display import clear_output
import time
import random
import copy

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# CONFIG
CONTACT = "<Nationstates demands that the User Agent contain a method of contacting the Script's owner. An email address is fine>"

In [ ]:
# setup
api = ns.Nationstates("Instance of https://github.com/bekaertruben/ns-census-maximizer/tools [contact: {}]".format(CONTACT))
world = api.world()

nations = world.get_shards("nations").nations.split(",")

In [ ]:
# get census scores for 10000 random nations (this takes two hours or more, lower this number to make it go quicker)
num = 10000

all_scores = {i:np.array([]) for i in range(87)}

max_bars = 100

nations_to_inspect = copy.copy(nations)
random.shuffle(nations_to_inspect)
nations_to_inspect = nations_to_inspect[:num]
for i in range(len(nations_to_inspect)):
    progress = (i+1)/num
    percent = int(1000 * progress)/10
    bars = int(max_bars * progress)
    clear_output()
    print("{}% - [{}{}]".format(percent, bars*"█", (max_bars-bars)*" "))

    name = nations_to_inspect[i]
    try:
        scores = api.nation(name).get_shards(ns.Shard("census", scale=list(range(87)), mode="score")).census.scale
        for scale in scores:
            all_scores[int(scale.id)] = np.append(all_scores[int(scale.id)], [float(scale.score)])
    except BaseException as e:
        print("nation \"{}\" raised <{}>".format(name, e))
    time.sleep(30/45)
print("DONE!")

In [ ]:
# plot a histogram for one of the scales to check whether this came out sensibly
plt.figure(figsize=(18, 6))
plt.hist(all_scores[0], bins=20)
plt.show()

In [ ]:
# process the data
means = dict()
stds = dict()
for i in all_scores:
    means[i] = np.mean(all_scores[i])
    stds[i] = np.std(all_scores[i])

In [ ]:
# save to file
with open("census_distribution.txt", "w") as file:
    file.writelines(["{ # id: (mean, std)\n"] + [
        "    {}: ({}, {}),\n".format(c_id, means[c_id], stds[c_id])
        for c_id in range(87)
    ] + ["}"])